In [1]:
from gensim.models import word2vec
import pandas as pd

In [2]:
#data preparation
data_df = pd.read_csv('order_products__train.csv')

data_dict = {}

for index,row in data_df.iterrows():
    if row['order_id'] not in data_dict.keys():
        data_dict[row['order_id']] = [str(row['product_id'])]
    else:
        data_dict[row['order_id']].append(str(row['product_id']))

    

In [3]:
data_list = [v for _,v in data_dict.items()]


In [4]:
model = word2vec.Word2Vec(data_list, size = 300, min_count = 1, workers = 10)

model.train(data_list, total_examples = len(data_list), epochs = 20)

(26667888, 27692340)

In [5]:
model.wv.most_similar(positive='19204') 
# input:
# 19204 - organic flakes

# results:
# 4410 - organic flakes
# 18224 - organic yogurt
# 25767 - organic cereal
# 340 - eco cereal
# 48007 - cereal
# 49241 - organic oatmeal
# 36914 - organic juice
# 49111 - oatmeal
# 32648 - gluten free granola
# 10831 - organic yogurt

/home/pfalkiewicz/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('18224', 0.6376736164093018),
 ('4410', 0.6199790239334106),
 ('340', 0.5811011791229248),
 ('34019', 0.5753176808357239),
 ('48007', 0.5563435554504395),
 ('36914', 0.5557258725166321),
 ('32585', 0.5494630336761475),
 ('49111', 0.5479534864425659),
 ('25767', 0.5449913740158081),
 ('44815', 0.538989782333374)]

In [9]:
import math
import numpy as np
def power_means(list_of_vectors):
    temp1,temp2,temp3 = [],[],[]
    for it in range(len(list_of_vectors[0])):
        tempC1 = 0
        tempC2 = []
        tempC3 = 0
        for vector in list_of_vectors:
            tempC1 += vector[it]
            tempC2.append(vector[it])
            tempC3 += 1/vector[it]
        temp1.append(tempC1/len(vector))
        temp2.append((np.sign(tempC2)*(np.abs(np.prod(tempC2)))**(1/len(tempC2)))[0])
        temp3.append(len(vector)/tempC3)
    
    return temp1 + temp2 + temp3
            
print(power_means([[0.0021,0.05,0.121],[0.21,0.001,0.1]]))

[0.0707, 0.017, 0.07366666666666667, 0.020999999999999998, 0.007071067811865475, 0.11, 0.006237623762376237, 0.0029411764705882353, 0.16425339366515837]


In [10]:
data_dict_pm = {}

for transaction_key in data_dict:
    data_dict_pm[transaction_key] = power_means([model[product] for product in data_dict[transaction_key]])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [23]:
# save vectors
col_key = [k for k,v in data_dict.items()]
col_value = [data_dict_pm[key] for key in col_key]
transactions_df = pd.DataFrame({'order_id':col_key, 'order_vector': col_value})
transactions_df.to_csv('transactions_vectors.csv', index = None)

In [28]:
#generate sample queries
import random
length_of_queries = 100
sample_query_uni = [random.choice(random.choice([v for _,v in data_dict.items()])) for i in range(length_of_queries)]

# A - products from one query
# B - products from different queries

sample_query_bigramA = []
for i in range(length_of_queries):
    temp = []
    transaction = random.choice([v for v in data_list if len(v) > 1])
    temp.append(random.choice(transaction))
    temp.append(random.choice(transaction))
    sample_query_bigramA.append(temp)
    
sample_query_bigramB = []
for i in range(length_of_queries):
    temp = []
    transaction1 = random.choice([v for v in data_list if len(v) > 1])
    transaction2 = random.choice([v for v in data_list if len(v) > 1])
    temp.append(random.choice(transaction1))
    temp.append(random.choice(transaction2))
    sample_query_bigramB.append(temp)
        
sample_query_trigramA = []
for i in range(length_of_queries):
    temp = []
    transaction = random.choice([v for v in data_list if len(v) > 2])
    temp.append(random.choice(transaction))
    temp.append(random.choice(transaction))
    temp.append(random.choice(transaction))
    sample_query_trigramA.append(temp)
    
sample_query_trigramB = []
for i in range(length_of_queries):
    temp = []
    transaction1 = random.choice([v for v in data_list if len(v) > 2])
    transaction2 = random.choice([v for v in data_list if len(v) > 2])
    transaction3 = random.choice([v for v in data_list if len(v) > 2])
    temp.append(random.choice(transaction1))
    temp.append(random.choice(transaction2))
    temp.append(random.choice(transaction3))
    sample_query_trigramB.append(temp)

In [32]:
queries = {}

for query in sample_query_uni:
    queries[query] = power_means([model[query]])
    
for query in sample_query_bigramA:
    queries[str(query)] = power_means([model[element] for element in query])

for query in sample_query_bigramB:
    queries[str(query)] = power_means([model[element] for element in query])    
    
for query in sample_query_trigramA:
    queries[str(query)] = power_means([model[element] for element in query])    
    
for query in sample_query_trigramA:
    queries[str(query)] = power_means([model[element] for element in query])    

    
for k,v in queries.items():
    print(k)
    print(v)
    break

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method

['31343', '13176', '17044']
[0.00015939762194951376, 0.0023361361026763916, 0.010793394645055136, -0.004086797833442688, -0.002732559194167455, 0.006671024362246195, 0.0024047722915808358, -0.005743198295434316, 0.008736459910869599, 0.0006634520987669627, -0.0015268316368261972, 0.0004138262073198954, -0.002053177207708359, 0.00047421445449193317, 0.00104899396498998, 0.0058584028482437135, 0.0019116647541522979, -0.004391050239404042, 0.007391178409258525, 0.0008837867590288321, -0.005324080636103948, 0.0007252286871274312, 0.005200568387905757, -0.006218385001023611, 0.0006912015378475189, -0.005020518352588018, 0.004673641597231229, -0.0044468155999978385, -0.008504928648471832, -0.008678463300069173, 0.002706826130549113, 0.0018314675490061442, 0.00035948436707258227, 0.0030539511144161225, -0.0015557433664798736, 0.0029619341591993966, 0.0002699337899684906, 0.007151846985022227, 9.004453818003337e-05, 0.00024917376538117727, 0.003639169236024221, 0.0017639837662378947, -0.001981